# Example: Impact Fund Screening
In this example we score potential land acquisitions in batch. The workflow shows how to combine land-cover metrics, B-Score calculation and occurrence density.

In [ ]:
from pathlib import Path
import pandas as pd
from verdesat.geo.aoi import AOI
from verdesat.services.bscore import compute_bscores
from verdesat.biodiv.gbif_validator import OccurrenceService


In [ ]:
# Load AOIs from the GeoJSON defined in the previous notebook
projects = eval(Path('examples/01_Reforestation_Analysis.ipynb').read_text().split('projects = ')[1].split('
')[0])
aois = AOI.from_geojson(projects, id_col='ID2')

In [ ]:
df_scores = compute_bscores(Path('examples')/'aoi.geojson' if False else projects, year=2021)
df_scores['id'] = df_scores['id'].astype(int)
print(df_scores)

In [ ]:
occ = OccurrenceService()
# dens = [occ.occurrence_density_km2(occ.fetch_occurrences(aoi), aoi.geometry.area*111**2) for aoi in aois]
